In [1]:
import math
from operator import itemgetter


#读取电影数据
def load_movielens(path='./movielens/ml-100k'):
    # get movie titles
    movies = {}
    for line in open(path + '/u.item', encoding='latin-1'):
        id, title = line.split('|')[0:2]
        movies[id] = title
    # load data
    prefs = {}
    for line in open(path + '/u.data', encoding='latin-1'):
        user, movieid, rating, ts = line.split('\t')
        prefs.setdefault(user, {})
        prefs[user][movies[movieid]] = float(rating)
    return prefs


def itemsim(prefs):
    user_item = {}
    for user, item in prefs.items():
        for i in item.keys():
            if user not in user_item.keys():
                user_item[user] = set()
            user_item[user].add(i)
    c = dict()
    d = dict()
    for user, item in user_item.items():
        for i in item:
            if i not in d.keys():
                d[i] = 0
            d[i] += 1
            for j in item:
                if i == j:
                    continue
                if (i,j) not in c.keys():
                    c[i,j] = 0
                c[i,j] += 1
    w = {}
    for items, cuv in c.items():
        w[items] = cuv / math.sqrt(d[items[0]] * d[items[1]])
    return w
        
def match(user, dicc, w, k):
    rvi = 1
    rank = {}
    related_user= []
    interacted_items = dicc[user].keys()
    for co_user, item in w.items():
        if co_user[0] == user:
            related_user.append((co_user[1], item))
    for v, wuv in sorted(related_user, key=itemgetter(1), reverse=True)[0: k]:
        for i in dicc[v]:
            if i in dicc[v]:
                if i in interacted_items:
                    continue
                if i not in rank.keys():
                    rank[i] = 0
                rank[i] += wuv * rvi
    return rank

prefs = load_movielens()
w = itemsim(prefs)
last = match('176', prefs, w, 4)
